In [19]:
#encoding: UTF-8 
import os
import numpy as np
import pandas as pd
import jieba
import jieba.analyse
import codecs
import jieba.posseg as pseg 
import json

# Basic Words processing

In [2]:
from collections import Counter
import jieba
jieba.load_userdict('userdict.txt')

# 创建停用词list
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r',encoding='utf-8').readlines()]
    return stopwords

# 对句子进行分词
def seg_sentence(sentence):
    sentence_seged = jieba.cut(sentence.strip())
    stopwords = stopwordslist('stoped.txt')  # 这里加载停用词的路径
    outstr = ''
    for word in sentence_seged:
        if word not in stopwords:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/89/qp8kb90s49z7xcdskq2c0qlm0000gn/T/jieba.cache
Loading model cost 0.645 seconds.
Prefix dict has been built succesfully.


In [3]:
# 拆分句子
# 版本为python3，如果为python2需要在字符串前面加上u
import re
def cut_sent(para):
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)  # 单字符断句符
    para = re.sub('([，！？\?])([^”’])', r"\1\n\2", para)  # 单字符断句符
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para)  # 英文省略号
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  # 中文省略号
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    
    # 如果双引号前有终止符，那么双引号才是句子的终点，把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
    para = para.rstrip()  # 段尾如果有多余的\n就去掉它
    # 很多规则中会考虑分号;，但是这里我把它忽略不计，破折号、英文双引号等同样忽略，需要的再做些简单调整即可。
    return para.split("\n")


#  Emotion corpus reading

In [4]:
em_df = pd.read_excel('情感词典修改版.xlsx')

In [5]:
em_dict = em_df[['词语','情感分类']].set_index('词语').T.to_dict(orient="list")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


In [6]:
#emotion_words = list(em_df['词语'])

# Emotion data processing

In [7]:
def emotions_analyze(file_path, em_dict):
    titles = []
    result_emotion = []
    result_words = []
    text = open(file_path, 'r',encoding="utf-8",errors='ignore') #加载要处理的文件的路径
    l = cut_sent(text.read())
    text.close()
    #print(l)
    #for line in text: #这个循环是遍历整个文本，inputs是整个文本，line是文本中的每一段
        #l=cut_sent(line)   # l是给一段文本进行拆分句子后的结果
    for w in l:  #这个循环是遍历一段文字里的每个句子,l是一段里的所有句子存放的list，w是每个句子
        line_seg = seg_sentence(w)  #line_seg 是一个句子拆分词语后的结果
        words=list(line_seg.split(' ')) #将一个句子中的词语拆分后逐个存放在list中
        for w in words:
            if w in list(em_dict):
                result_words.append(w) #筛选出情感词
                result_emotion.append(em_dict[w][0]) #匹配情感
                #print(em_dict[w])
                
    return result_words, result_emotion

# Files reading

In [8]:
file_dir = './dataset/emotional_words_data_vis/stories/'
files = os.listdir(file_dir)

# Results generating

In [49]:
def emotion_results_process(file, result_emotion):
    result_df = pd.DataFrame(data = result_emotion)
    result_df.rename(columns={0:'emotions'}, inplace = True)    
    # .size() 聚合统计 .reset_index() 把index变成正常的列
    #print(result_df)
    new_df = result_df.groupby('emotions').size().reset_index()
    new_df.rename(columns={0: 'Times'}, inplace = True) 
    #new_df['Times'] = new_df['Times'].map(lambda x: x*10) #10是为了可视化数据需要进行的一个处理
    return new_df

In [50]:
data_for_vis = []
temp_dicts = {}
emotion_unique_df = pd.DataFrame()
for file in files:
    if file == '.DS_Store':
        continue
    print(file)
    title = file.split('.txt')[0]
    file_path = file_dir+file
    result_words, result_emotion = emotions_analyze(file_path, em_dict)
    emotion_df = emotion_results_process(file, result_emotion)
    temp_dicts[title] = emotion_df
    emotion_unique_df[title] = emotion_df['emotions']
    


乌木马的故事.txt
海姑娘和她儿子的故事.txt
苏尔克和伊波丽莎.txt
银匠的神奇经历.txt
麦仑.txt
孔马康出走他乡.txt
渔翁、魔鬼和比目鱼的故事.txt
阿拉丁和灯神的故事(续).txt
宰曼和白杜尔.txt
阿里巴巴和四十大盗.txt
睡着的国王的故事.txt
狐狸和狼.txt


# keep the matches (process this part of data for a emotional data visualization)

In [51]:
result_df = emotion_unique_df.dropna()

In [52]:
result_df.to_excel('./dataset/emotional_words_data_vis/emotions.xls')

In [53]:
print(list(result_df[result_df.columns[0]]))

['NB', 'NC', 'ND', 'NE', 'NG', 'NH', 'NI', 'NJ', 'NL', 'NN', 'PA', 'PB', 'PC', 'PD', 'PE', 'PF', 'PG', 'PH', 'PH ']


In [64]:
emotions = ['NB', 'NC', 'ND', 'NE', 'NG', 'NH', 'NI', 'NJ', 'NL', 'NN', 
            'PA', 'PB', 'PC', 'PD', 'PE', 'PF', 'PG','PK']

In [112]:
def data_vis_data_process(title, emotion_df):
    data_vis_dict={}
    
    emo_df_trans= emotion_df.set_index('emotions').T.drop(['PH','PH ','NN','NJ'], axis=1)
    #print(emo_df_trans)
    pie_chart_dict = emo_df_trans.to_dict('int')# prepare data for pie chart
    #print(pie_chart_dict)
    #print(title)
    data_vis_dict['Title'] = title
    data_vis_dict['Content'] = list(pie_chart_dict.values())[0]
    print(data_vis_dict)
    return data_vis_dict

In [113]:
data_for_vis = []
for title in temp_dicts.keys():
    print(title)
    emotion_df = temp_dicts[title]
    #print(emotion_df)
    data_vis_dict = data_vis_data_process(title, emotion_df)
    data_for_vis.append(data_vis_dict)

乌木马的故事
{'Title': '乌木马的故事', 'Content': {'NB': 29, 'NC': 10, 'ND': 24, 'NE': 23, 'NG': 1, 'NH': 7, 'NI': 13, 'NL': 6, 'PA': 67, 'PB': 15, 'PC': 17, 'PD': 21, 'PE': 13, 'PF': 7, 'PG': 23, 'PK': 28}}
海姑娘和她儿子的故事
{'Title': '海姑娘和她儿子的故事', 'Content': {'NB': 49, 'NC': 23, 'ND': 25, 'NE': 39, 'NG': 3, 'NH': 2, 'NI': 17, 'NL': 12, 'PA': 105, 'PB': 82, 'PC': 25, 'PD': 51, 'PE': 25, 'PF': 16, 'PG': 41, 'PK': 44}}
苏尔克和伊波丽莎
{'Title': '苏尔克和伊波丽莎', 'Content': {'NB': 55, 'NC': 21, 'ND': 67, 'NE': 42, 'NG': 10, 'NH': 8, 'NI': 10, 'NL': 29, 'PA': 85, 'PB': 50, 'PC': 25, 'PD': 51, 'PE': 21, 'PF': 11, 'PG': 42, 'PK': 19}}
银匠的神奇经历
{'Title': '银匠的神奇经历', 'Content': {'NB': 66, 'NC': 30, 'ND': 42, 'NE': 46, 'NH': 8, 'NI': 15, 'NK': 1, 'NL': 12, 'PA': 128, 'PB': 48, 'PC': 31, 'PD': 34, 'PE': 30, 'PF': 11, 'PG': 42, 'PK': 22}}
麦仑
{'Title': '麦仑', 'Content': {'NB': 43, 'NC': 9, 'ND': 26, 'NE': 33, 'NG': 4, 'NH': 5, 'NI': 15, 'NK': 2, 'NL': 16, 'PA': 78, 'PB': 65, 'PC': 9, 'PD': 27, 'PE': 12, 'PF': 11, 'PG': 26, 'PK': 2

In [114]:
print(data_for_vis)

[{'Title': '乌木马的故事', 'Content': {'NB': 29, 'NC': 10, 'ND': 24, 'NE': 23, 'NG': 1, 'NH': 7, 'NI': 13, 'NL': 6, 'PA': 67, 'PB': 15, 'PC': 17, 'PD': 21, 'PE': 13, 'PF': 7, 'PG': 23, 'PK': 28}}, {'Title': '海姑娘和她儿子的故事', 'Content': {'NB': 49, 'NC': 23, 'ND': 25, 'NE': 39, 'NG': 3, 'NH': 2, 'NI': 17, 'NL': 12, 'PA': 105, 'PB': 82, 'PC': 25, 'PD': 51, 'PE': 25, 'PF': 16, 'PG': 41, 'PK': 44}}, {'Title': '苏尔克和伊波丽莎', 'Content': {'NB': 55, 'NC': 21, 'ND': 67, 'NE': 42, 'NG': 10, 'NH': 8, 'NI': 10, 'NL': 29, 'PA': 85, 'PB': 50, 'PC': 25, 'PD': 51, 'PE': 21, 'PF': 11, 'PG': 42, 'PK': 19}}, {'Title': '银匠的神奇经历', 'Content': {'NB': 66, 'NC': 30, 'ND': 42, 'NE': 46, 'NH': 8, 'NI': 15, 'NK': 1, 'NL': 12, 'PA': 128, 'PB': 48, 'PC': 31, 'PD': 34, 'PE': 30, 'PF': 11, 'PG': 42, 'PK': 22}}, {'Title': '麦仑', 'Content': {'NB': 43, 'NC': 9, 'ND': 26, 'NE': 33, 'NG': 4, 'NH': 5, 'NI': 15, 'NK': 2, 'NL': 16, 'PA': 78, 'PB': 65, 'PC': 9, 'PD': 27, 'PE': 12, 'PF': 11, 'PG': 26, 'PK': 21}}, {'Title': '孔马康出走他乡', 'Conten

In [115]:
json.dumps(data_for_vis,ensure_ascii=False)

'[{"Title": "乌木马的故事", "Content": {"NB": 29, "NC": 10, "ND": 24, "NE": 23, "NG": 1, "NH": 7, "NI": 13, "NL": 6, "PA": 67, "PB": 15, "PC": 17, "PD": 21, "PE": 13, "PF": 7, "PG": 23, "PK": 28}}, {"Title": "海姑娘和她儿子的故事", "Content": {"NB": 49, "NC": 23, "ND": 25, "NE": 39, "NG": 3, "NH": 2, "NI": 17, "NL": 12, "PA": 105, "PB": 82, "PC": 25, "PD": 51, "PE": 25, "PF": 16, "PG": 41, "PK": 44}}, {"Title": "苏尔克和伊波丽莎", "Content": {"NB": 55, "NC": 21, "ND": 67, "NE": 42, "NG": 10, "NH": 8, "NI": 10, "NL": 29, "PA": 85, "PB": 50, "PC": 25, "PD": 51, "PE": 21, "PF": 11, "PG": 42, "PK": 19}}, {"Title": "银匠的神奇经历", "Content": {"NB": 66, "NC": 30, "ND": 42, "NE": 46, "NH": 8, "NI": 15, "NK": 1, "NL": 12, "PA": 128, "PB": 48, "PC": 31, "PD": 34, "PE": 30, "PF": 11, "PG": 42, "PK": 22}}, {"Title": "麦仑", "Content": {"NB": 43, "NC": 9, "ND": 26, "NE": 33, "NG": 4, "NH": 5, "NI": 15, "NK": 2, "NL": 16, "PA": 78, "PB": 65, "PC": 9, "PD": 27, "PE": 12, "PF": 11, "PG": 26, "PK": 21}}, {"Title": "孔马康出走他乡", "Conte

In [116]:
d.Content.NB+d.Content.NC+d.Content.ND+d.Content.NE+d.Content.NG
+d.Content.NH+d.Content.NI+d.Content.NJ+d.Content.NL
+d.Content.PA+d.Content.PB+d.Content.PC+d.Content.PD+d.Content.PE
+d.Content.PF+d.Content.PG+d.Content.PK

NameError: name 'd' is not defined

In [ ]:
NB:"#82ccdd",NC:"#60a3bc",ND:"#3c6382",NE:"#0a3d62",NG:"#0c2461",
NH:"#1e3799",NI:"#4a69bd",NJ:"#6a89cc",NL:"#38ada9",
PA:"#f8c291",PB:"#e55039",PC:"#eb2f06",PD:"#b71540",PE:"#e58e26",
PF:"#fa983a",PG:"#fad390",PK:"#b8e994"


In [ ]:
'NB', 'NC', 'ND', 'NE', 'NG', 'NH', 'NI', 'NJ', 'NL', 
'PA', 'PB', 'PC', 'PD', 'PE', 'PF', 'PG','PK'

In [ ]:
{"NB": 29, "NC": 10, "ND": 24, "NE": 23, "NG": 1, 
 "NH": 7, "NI": 13, "NJ": 3, "NL": 6, 
 "PA": 67, "PB": 15, "PC": 17, "PD": 21, "PE": 13,
 "PF": 7, "PG": 23, "PH ": 17, "PK": 28}}